**Connect google drive**

In [ ]:
# To Check GPU
!nvidia-smi

Fri May 28 19:13:11 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   65C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# For mounting Drive to Google Colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
'Colab Notebooks'  'My Drive'   yolov3	 yolov4


**1) Clone the Darknet**



In [ ]:
# Clone the Darknet to the Colab
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15054, done.
remote: Total 15054 (delta 0), reused 0 (delta 0), pack-reused 15054
Receiving objects: 100% (15054/15054), 13.43 MiB | 16.37 MiB/s, done.
Resolving deltas: 100% (10234/10234), done.


**2) Compile Darknet using Nvidia GPU**


In [ ]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

**3) Configure Darknet network for training YOLO V3**

In [ ]:
# Configuration file (cfg file)
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [ ]:
# Setting for single object

!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

In [ ]:
# Create folder on google drive so that we can save there the weights
!mkdir "/mydrive/yolov3/"

mkdir: cannot create directory ‘/mydrive/yolov3/’: File exists


In [ ]:
# Emulator is a class name
!echo "Emulator" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3/' > data/obj.data
!mkdir data/obj

In [ ]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2021-05-28 19:15:00--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  67.6MB/s    in 2.3s    

2021-05-28 19:15:02 (67.6 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



**4) Extract Images**

The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" on Google Drive

In [ ]:
# Extracting custom object dataset
!unzip /mydrive/yolov3/Emulator.zip -d data/obj

In [ ]:
# We're going to convert the class index on the .txt files. As we're working with only one class, it's supposed to be class 0.
# If the index is different from 0 then we're going to change it.
import glob
import os
import re

txt_file_paths = glob.glob(r"data/obj/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
            if numbers:

                # Define coordinates
                text = "{} {} {} {} {}".format(0, numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
                print(i, file_path)
                print(text)
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

15 0.519792 0.466204 0.205208 0.476852

['15', '0.519792', '0.466204', '0.205208', '0.476852']
0 data/obj/picture_10_2.txt
0 0.519792 0.466204 0.205208 0.476852
15 0.538281 0.432407 0.215104 0.450000

['15', '0.538281', '0.432407', '0.215104', '0.450000']
1 data/obj/picture_18_2.txt
0 0.538281 0.432407 0.215104 0.450000
15 0.613021 0.354167 0.278125 0.293519

['15', '0.613021', '0.354167', '0.278125', '0.293519']
2 data/obj/picture_17_3.txt
0 0.613021 0.354167 0.278125 0.293519
15 0.504687 0.464815 0.206250 0.364815

['15', '0.504687', '0.464815', '0.206250', '0.364815']
3 data/obj/picture_2_0.txt
0 0.504687 0.464815 0.206250 0.364815
15 0.525260 0.475926 0.196354 0.500000

['15', '0.525260', '0.475926', '0.196354', '0.500000']
4 data/obj/picture_10_0.txt
0 0.525260 0.475926 0.196354 0.500000
15 0.434635 0.496759 0.271354 0.171296

['15', '0.434635', '0.496759', '0.271354', '0.171296']
5 data/obj/picture_16_0.txt
0 0.434635 0.496759 0.271354 0.171296
15 0.540365 0.428241 0.210938 0.445

In [ ]:
import glob
images_list = glob.glob("data/obj/*.png")
print(images_list)

['data/obj/picture_16_0.png', 'data/obj/picture_13_3.png', 'data/obj/picture_5_1.png', 'data/obj/picture_18_2.png', 'data/obj/picture_9_3.png', 'data/obj/picture_20_0.png', 'data/obj/picture_9_4.png', 'data/obj/picture_16_1.png', 'data/obj/picture_9_1.png', 'data/obj/picture_7_0.png', 'data/obj/picture_8_4.png', 'data/obj/picture_10_2.png', 'data/obj/picture_16_3.png', 'data/obj/picture_17_0.png', 'data/obj/picture_4_2.png', 'data/obj/picture_3_1.png', 'data/obj/picture_5_3.png', 'data/obj/picture_10_4.png', 'data/obj/picture_17_2.png', 'data/obj/picture_15_0.png', 'data/obj/picture_10_0.png', 'data/obj/picture_18_4.png', 'data/obj/picture_6_3.png', 'data/obj/picture_19_1.png', 'data/obj/picture_20_3.png', 'data/obj/picture_15_4.png', 'data/obj/picture_1_1.png', 'data/obj/picture_7_1.png', 'data/obj/picture_4_4.png', 'data/obj/picture_0_0.png', 'data/obj/picture_19_4.png', 'data/obj/picture_19_2.png', 'data/obj/picture_14_2.png', 'data/obj/picture_7_2.png', 'data/obj/picture_13_2.png',

In [ ]:
#Create training.txt file, required for YOLOv3 training
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

In [ ]:
# Calculates anchors for the custom dataset
!./darknet detector calc_anchors data/obj.data -num_of_clusters 9 -width 608 -height 608

 CUDA-version: 11000 (11020), cuDNN: 7.6.5, GPU count: 1  
 OpenCV version: 3.2.0

 num_of_clusters = 9, width = 608, height = 608 
 read labels from 105 images 
 loaded 	 image: 105 	 box: 105
 all loaded. 

 calculating k-means++ ...

 iterations = 8 


counters_per_class = 105

 avg IoU = 91.81 % 

Saving anchors to the file: anchors.txt 
anchors =  63,125, 161,105,  69,271,  93,270, 162,170, 104,281, 136,215, 129,276, 155,240



**6) Start the training**

In [ ]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.812766), count: 3, class_loss = 0.039368, iou_loss = 0.072138, total_loss = 0.111506 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.888291), count: 1, class_loss = 0.000002, iou_loss = 0.022775, total_loss = 0.022777 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
 total_bbox = 72649, rewritten_bbox = 0.000000 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.811720), count: 3, class_loss = 0.117932, iou_loss = 0.060473, total_loss = 0.178405 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.742008), count: 1, class_loss = 0.240415, iou_loss = 0.041338, total_loss = 0.281753 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.000000), count: 1, class_l

In [ ]:
# Training on last saved weights
!./darknet detector train data/obj.data cfg/yolov3_training.cfg '/content/gdrive/My Drive/yolov3/yolov3_training_last.weights' -dont_show

In [ ]:
# Testing on an image
!./darknet detect cfg/yolov3_testing.cfg '/content/gdrive/My Drive/yolov3/yolov3_training_final.weights' data/Test_img.jpg

 CUDA-version: 11000 (11020), cuDNN: 7.6.5, GPU count: 1  
 OpenCV version: 3.2.0
Couldn't open file: cfg/yolov3_testing.cfg


In [ ]:
# Mean Average Precision (mAP) Calculation
!./darknet detector map data/obj.data cfg/yolov3_training.cfg '/content/gdrive/My Drive/yolov3/yolov3_trained.weights'

 CUDA-version: 11000 (11020), cuDNN: 7.6.5, GPU count: 1  
 OpenCV version: 3.2.0
 0 : compute_capability = 750, cudnn_half = 0, GPU: Tesla T4 
net.optimized_memory = 0 
mini_batch = 1, batch = 16, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 Create CUDA-stream - 0 
 Create cudnn-handle 0 
conv     32       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  32 0.299 BF
   1 conv     64       3 x 3/ 2    416 x 416 x  32 ->  208 x 208 x  64 1.595 BF
   2 conv     32       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  32 0.177 BF
   3 conv     64       3 x 3/ 1    208 x 208 x  32 ->  208 x 208 x  64 1.595 BF
   4 Shortcut Layer: 1,  wt = 0, wn = 0, outputs: 208 x 208 x  64 0.003 BF
   5 conv    128       3 x 3/ 2    208 x 208 x  64 ->  104 x 104 x 128 1.595 BF
   6 conv     64       1 x 1/ 1    104 x 104 x 128 ->  104 x 104 x  64 0.177 BF
   7 conv    128       3 x 3/ 1    104 x 104 x  64 ->  104 x 104 x 128 1.595 BF
   8 Shortcut Layer: 5, 